<a href="https://colab.research.google.com/github/giangntgg/CourseProject/blob/main/03_Try_K_Means_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install -U torch
!pip install -U transformers
!pip install -U sentence-transformers
!pip install -U pandas
!pip install spacy==2.2.4
!python -m spacy download en

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\USER\AppData\Local\Programs\Python\Python37\lib\site-packages\en_core_web_sm
-->
C:\Users\USER\AppData\Local\Programs\Python\Python37\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


You do not have sufficient privilege to perform this operation.


In [26]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import spacy
import pandas as pd
import re
import implicit_skill_finder
from joblib import dump, load

In [27]:
# KEY IN A SENTENCE
sentences = ['Work on complex and extremely varied data sets from some of the world’s largest organisations to solve real world problems Develop data science products and solutions for clients as well as for our data science team Write highly optimized code to advance our internal Data Science Toolbox Work in a multi-disciplinary environment with specialists in machine learning, engineering and design Focus on modelling by working alongside the Data Engineering team Add real-world impact to your academic expertise, as you are encouraged to write papers and present at meetings and conferences should you wish Take part in R&D (video: R&D at QuantumBlack); attend conferences such as NIPS and ICML as well as data science retrospectives where you will have the opportunity to share and learn from your co-workers Work in one of the most advanced data science teams globally']
related_docs = [implicit_skill_finder.find_similar_texts([sentence]) for sentence in sentences]

In [28]:
def clean_text(texts):
  # remove entering 
  texts = [i.replace('\n', '').replace('\r', '') for i in texts]
  print('Remove entering...')

  # remove URL's 
  texts = [re.sub(r'http\S+', '', i) for i in texts]
  print('Remove HTTPS...')

  # convert text to lowercase
  texts = [i.lower() for i in texts]
  print('Lowercasing...')

  # remove numbers
  texts = [i.replace("[0-9]", " ") for i in texts]
  print('Remove numbers...')

  # remove whitespaces
  texts = [' '.join(i.split()) for i in texts]
  print('Remove whitespaces...')

  # remove empty tokens
  texts = [''.join(ch for ch in i if len(ch) > 0) for i in texts]
  print('Remove empty tokens...')

  return texts 

# import spaCy's language model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

sentences = clean_text(sentences)
sentences = lemmatization(sentences)
related_docs = [clean_text(docs) for docs in related_docs]
related_docs = [lemmatization(docs) for docs in related_docs]

Remove entering...
Remove HTTPS...
Lowercasing...
Remove numbers...
Remove whitespaces...
Remove empty tokens...
Remove entering...
Remove HTTPS...
Lowercasing...
Remove numbers...
Remove whitespaces...
Remove empty tokens...


In [29]:
tfidf = load('TFIDF_Vectorizer.joblib')
tfidf_docs = tfidf.transform(sentences)

In [30]:
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
bert_docs = embedder.encode(sentences)

Load the pre-text model

In [31]:
tfidf_model = load('TFIDF_JobClustering (2).joblib')
bert_model = load('BERT_JobClustering.joblib')

Get the cluster result

In [32]:
bert_label = bert_model.predict(bert_docs)
tfidf_label = tfidf_model.predict(tfidf_docs)

Get the skills that are relevant

In [33]:
bert_skills = pd.read_csv('BERT Skill Group.csv')
bert_skills = bert_skills.fillna('NA')
tfidf_skills = pd.read_csv('TFIDF Skill Group.csv')
tfidf_skills = tfidf_skills.fillna('NA')
current_skills = set()
implicit_skills = set()


for i in range(len(bert_label)):
  cluster = bert_label[i]
  print('---- BERT-based Skills Extractor----')
  print(f'Trending skills for job description {i}: ' + bert_skills[bert_skills['BERT Cluster'] == cluster]['jobSkill_y'].iloc[0])
  print(f'Relevant industry for job description {i}: ' + bert_skills[bert_skills['BERT Cluster'] == cluster]['jobSkill_x'].iloc[0])
  current_skills.clear()
  implicit_skills.clear()
  current_skills.update(bert_skills[bert_skills['BERT Cluster'] == cluster]['jobSkill_y'].iloc[0].split(", "))
  current_skills.update(bert_skills[bert_skills['BERT Cluster'] == cluster]['jobSkill_x'].iloc[0].split(", "))
  implicit_bert_docs = embedder.encode(related_docs[i])
  implicit_bert_label = bert_model.predict(implicit_bert_docs)
  for j in range(len(implicit_bert_label)):
    implicit_cluster = implicit_bert_label[j]
    implicit_skills.update(bert_skills[bert_skills['BERT Cluster'] == cluster]['jobSkill_y'].iloc[0].split(", "))
    implicit_skills.update(bert_skills[bert_skills['BERT Cluster'] == cluster]['jobSkill_x'].iloc[0].split(", "))
  implicit_skills = implicit_skills.difference(current_skills)
  print(f'Implicit skills for job description {i}: ' + ', '.join(implicit_skills))


for i in range(len(tfidf_label)):
  cluster = tfidf_label[i]
  print('---- TFIDF-based Skills Extractor----')
  print(f'Trending skills for job description {i}: ' + tfidf_skills[tfidf_skills['TFIDF Cluster'] == cluster]['jobSkill_y'].iloc[0])
  print(f'Relevant industry for job description {i}: ' + tfidf_skills[tfidf_skills['TFIDF Cluster'] == cluster]['jobSkill_x'].iloc[0])
  current_skills.clear()
  implicit_skills.clear()
  current_skills.update(tfidf_skills[tfidf_skills['TFIDF Cluster'] == cluster]['jobSkill_y'].iloc[0].split(", "))
  current_skills.update(tfidf_skills[tfidf_skills['TFIDF Cluster'] == cluster]['jobSkill_x'].iloc[0].split(", "))
  implicit_tfidf_docs = tfidf.transform(related_docs[i])
  implicit_tfidf_label = tfidf_model.predict(implicit_tfidf_docs)
  for j in range(len(implicit_tfidf_label)):
    implicit_cluster = implicit_tfidf_label[j]
    implicit_skills.update(tfidf_skills[tfidf_skills['TFIDF Cluster'] == cluster]['jobSkill_y'].iloc[0].split(", "))
    implicit_skills.update(tfidf_skills[tfidf_skills['TFIDF Cluster'] == cluster]['jobSkill_x'].iloc[0].split(", "))
  implicit_skills = implicit_skills.difference(current_skills)
  print(f'Implicit skills for job description {i}: ' + ', '.join(implicit_skills))


---- BERT-based Skills Extractor----
Trending skills for job description 0: NA
Relevant industry for job description 0: Engineering, Information Technology,Information Technology,Analyst, Information Technology, Engineering,Quality Assurance, Engineering, Information Technology,Information Technology, Product Management, Engineering,Information Technology, Consulting, Engineering,Engineering,Other,Information Technology, Engineering,Other, Information Technology, Management,Human Resources,Finance, Sales,Management, Manufacturing,Marketing, Sales,Engineering, Information Technology, Research,Engineering, Manufacturing, Design
Implicit skills for job description 0: 
---- TFIDF-based Skills Extractor----
Trending skills for job description 0: Data mining software
Relevant industry for job description 0: Information Technology, Product Management, Engineering,Engineering, Information Technology,Information Technology,Information Technology, Business Development,Finance, Information Techno